In [2]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("red-wine-quality-prediction")

<Experiment: artifact_location='s3://ashish-mlops-project-run-1/1', experiment_id='1', lifecycle_stage='active', name='red-wine-quality-prediction', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_csv(filename)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    numerical = ['total sulfur dioxide', 'free sulfur dioxide', 'alcohol', 'volatile acidity']
    dicts = df[numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('data/red_wine_train.csv')
df_val = read_dataframe('data/red_wine_val.csv')

target = 'quality'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 0.6994300162237136


/home/ubuntu/.local/share/virtualenvs/07-web-service-mlflow-3ExaZQF4/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
